In [11]:
PATH_FILE_PREPROCESSED = './results/ds.parquet'
PATH_FILE_ELAPSE_TIME = './results/elapse_time.csv'
PATH_PREFIX_MODEL_LDA = './results/models_lda/'
PATH_PREFIX_MODEL_BERTOPIC = './results/models_bertopic/'
LDA_N_TRIAL = 2

In [12]:
import pandas as pd

ds = pd.read_parquet(PATH_FILE_PREPROCESSED)
elapse_time = pd.read_csv(PATH_FILE_ELAPSE_TIME)

# LDA

In [13]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

def lda(docs):
    docs = docs.dropna()
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=LDA_N_TRIAL)

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

In [14]:
from tqdm import tqdm

def save_models_lda():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'B' in v: 
            r_time.append(0)
            continue
        model = lda(ds[v])
        model['model'].save(f'{PATH_PREFIX_MODEL_LDA}{v}')
        r_time.append(model['time'])
    elapse_time['lda_training'] = pd.Series(r_time)

In [ ]:
save_models_lda()

In [16]:
elapse_time

,variant,nlp,preprocessing,lda_training
0,dLWN,267.22045,0.790498,13.949261
1,DLWN,267.22045,1.186545,12.185707


# BERTopic

In [17]:
from time import time

from bertopic import BERTopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

def save_models_bertopic():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'T' in v: 
            r_time.append(0)
            continue
        t_start = time()
        docs = ds[v].dropna()
        docs = [' '.join(doc) for doc in docs] if 'B' not in v else docs
        bertopic = BERTopic(language='multilingual', embedding_model=embedding_model)
        model = bertopic.fit(docs)
        t_end = time()
        model.save(
            f"{PATH_PREFIX_MODEL_BERTOPIC}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    elapse_time['bertopic_training'] = pd.Series(r_time)

In [18]:
save_models_bertopic()

Training dLWN:   0%|          | 0/2 [00:00<?, ?it/s]

Training DLWN: 100%|██████████| 2/2 [05:15<00:00, 157.93s/it]


In [19]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training
0,dLWN,267.22045,0.790498,13.949261,285.131878
1,DLWN,267.22045,1.186545,12.185707,30.661990


In [20]:
elapse_time.to_csv(PATH_FILE_ELAPSE_TIME, index=False)